In [1]:
import pandas as pd
population = pd.read_csv('C:/Users/akosciansky/Desktop/Bokeh/population.csv')
life = pd.read_csv('C:/Users/akosciansky/Desktop/Bokeh/life.csv')
fertility = pd.read_csv('C:/Users/akosciansky/Desktop/Bokeh/fertility.csv')

In [2]:
life_melt = pd.melt(life, id_vars='Life expectancy', value_name='Life Expectancy Years', var_name='Year')
life_melt.rename(columns={'Life expectancy': 'Country'}, inplace=True)

In [3]:
population_melt = pd.melt(population, id_vars='Total population', value_name='Population', var_name='Year')
population_melt.rename(columns={'Total population': 'Country'}, inplace=True)

In [4]:
fertility_melt = pd.melt(fertility, id_vars='Total fertility rate', value_name='Fertility Rate', var_name='Year')
fertility_melt.rename(columns={'Total fertility rate': 'Country'}, inplace=True)

In [5]:
population_melt['Population'] = population_melt['Population'].str.replace(',', '')
population_melt['Population'] = population_melt['Population'].astype('float')

In [6]:
data = pd.merge(life_melt, fertility_melt, on=['Country', 'Year'], how='left')
data = pd.merge(data, population_melt, on=['Country', 'Year'], how='left')
data.head()

,Country,Year,Life Expectancy Years,Fertility Rate,Population
0,Abkhazia,1800,NaN,NaN,NaN
1,Afghanistan,1800,28.21,7.00,3280000.0
2,Akrotiri and Dhekelia,1800,NaN,NaN,NaN
3,Albania,1800,35.40,4.60,410445.0
4,Algeria,1800,28.82,6.99,2503218.0


In [7]:
data.columns = ['Country', 'year', 'life', 'fertility', 'population']
data.head()

,Country,year,life,fertility,population
0,Abkhazia,1800,NaN,NaN,NaN
1,Afghanistan,1800,28.21,7.00,3280000.0
2,Akrotiri and Dhekelia,1800,NaN,NaN,NaN
3,Albania,1800,35.40,4.60,410445.0
4,Algeria,1800,28.82,6.99,2503218.0


In [8]:
data = data.dropna(how='any')
data.head()

,Country,year,life,fertility,population
1,Afghanistan,1800,28.21,7.00,3280000.0
3,Albania,1800,35.40,4.60,410445.0
4,Algeria,1800,28.82,6.99,2503218.0
7,Angola,1800,26.98,6.93,1567028.0
9,Antigua and Barbuda,1800,33.54,5.00,37000.0


In [9]:
# Perform necessary imports
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import curdoc
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row
from bokeh.models import Slider

from bokeh.models import Select
from bokeh.models import CheckboxGroup, RadioGroup, Toggle


from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

In [10]:



# Make the ColumnDataSource: source
source = ColumnDataSource(data={
    'x'       : data.loc[data['year']=='1970'].fertility,
    'y'       : data.loc[data['year']=='1970'].life,
    'country'      : data.loc[data['year']=='1970'].Country,
    'pop'      : (data.loc[data['year']=='1970'].population / 20000000) + 2

})

In [11]:

# Create the Document Application
#def modify_doc(doc):

    # Save the minimum and maximum values of the fertility column: xmin, xmax
    xmin, xmax = min(data.fertility), max(data.fertility)

    # Save the minimum and maximum values of the life expectancy column: ymin, ymax
    ymin, ymax = min(data.life), max(data.life)

    # Create the figure: plot
    plot = figure(title='Gapminder Data for 1970', plot_height=400, plot_width=700,
                  x_range=(xmin, xmax), y_range=(ymin, ymax))

    # Make a list of the unique values from the region column: regions_list
    #regions_list = data.region.unique().tolist()

    # Make a color mapper: color_mapper
    #color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)


    # Add circle glyphs to the plot
    plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
                color=dict(field='region'), legend='region')

    # Set the x-axis label
    plot.xaxis.axis_label ='Fertility (children per woman)'

    # Set the y-axis label
    plot.yaxis.axis_label = 'Life Expectancy (years)'

    # Set the legend.location attribute of the plot to 'top_right'
    plot.legend.location = 'top_right'


    # Create a HoverTool: hover
    hover = HoverTool(tooltips=[('Country', '@country')])

    # Add the HoverTool to the plot
    plot.add_tools(hover)




    # Define the callback function: update_plot
    def update_plot(attr, old, new):
        # Read the current value off the slider and 2 dropdowns: yr, x, y
        yr = slider.value
        x = x_select.value
        y = y_select.value
        # Label axes of plot
        plot.xaxis.axis_label = x
        plot.yaxis.axis_label = y
        new_data = {
            'x'       : data.loc[yr].fertility,
            'y'       : data.loc[yr].life,
            'country' : data.loc[yr].Country,
            'pop'     : (data.loc[yr].population / 20000000) + 2
        }
        # Assign new_data to: source.data
        source_data = new_data

        # Set the range of all axes
        plot.x_range.start = min(data[x])
        plot.x_range.end = max(data[x])
        plot.y_range.start = min(data[y])
        plot.y_range.end = max(data[y])

        # Add title to figure: plot.title.text
        plot.title.text = 'Gapminder data for %d' % yr


    # Make a slider object: slider
    slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')

    # Attach the callback to the 'value' property of slider
    slider.on_change('value', update_plot)



    # Create a dropdown Select widget for the x data: x_select
    x_select = Select(
        options=['fertility', 'life', 'child_mortality', 'gdp'],
        value='fertility',
        title='x-axis data'
    )

    # Attach the update_plot callback to the 'value' property of x_select
    x_select.on_change('value', update_plot)

    # Create a dropdown Select widget for the y data: y_select
    y_select = Select(
        options=['fertility', 'life', 'child_mortality', 'gdp'],
        value='life',
        title='y-axis data'
    )

    # Attach the update_plot callback to the 'value' property of y_select
    y_select.on_change('value', update_plot)



    # Make a row layout of widgetbox(slider) and plot and add it to the current document
    layout = row(widgetbox(slider, x_select, y_select), plot)

    # Add the plot to the current document and add a title
    curdoc().add_root(layout)
    curdoc().title = 'Gapminder'
    
    #Show layout
    show(layout)
    
# Set up the Application 
#handler = FunctionHandler(modify_doc)
#app = Application(handler)

IndentationError: unexpected indent (<ipython-input-11-8c55adce9e1e>, line 6)

In [14]:
# Create the Document
# Not strictly necessary, but helps w/ debugging
doc = app.create_document()

In [18]:
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")

AttributeError: 'State' object has no attribute '_notebook_type'

In [13]:
# Save the minimum and maximum values of the fertility column: xmin, xmax
xmin, xmax = min(data.fertility), max(data.fertility)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(data.life), max(data.life)

# Create the figure: plot
plot = figure(title='Gapminder Data for 1970', plot_height=400, plot_width=700,
              x_range=(xmin, xmax), y_range=(ymin, ymax))

# Make a list of the unique values from the region column: regions_list
#regions_list = data.region.unique().tolist()

# Make a color mapper: color_mapper
#color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)


# Add circle glyphs to the plot
plot.circle(x='x', y='y', fill_alpha=0.8, source=source)

# Set the x-axis label
plot.xaxis.axis_label ='Fertility (children per woman)'

# Set the y-axis label
plot.yaxis.axis_label = 'Life Expectancy (years)'

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'


# Create a HoverTool: hover
hover = HoverTool(tooltips=[('Country', '@country')])

# Add the HoverTool to the plot
plot.add_tools(hover)




# Define the callback function: update_plot
def update_plot(attr, old, new):
    # Read the current value off the slider and 2 dropdowns: yr, x, y
    yr = slider.value
    x = x_select.value
    y = y_select.value
    # Label axes of plot
    plot.xaxis.axis_label = x
    plot.yaxis.axis_label = y
    new_data = {
        'x'       : data.loc[yr].fertility,
        'y'       : data.loc[yr].life,
        'country' : data.loc[yr].Country,
        'pop'     : (data.loc[yr].population / 20000000) + 2
    }
    # Assign new_data to: source.data
    source_data = new_data

    # Set the range of all axes
    plot.x_range.start = min(data[x])
    plot.x_range.end = max(data[x])
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])

    # Add title to figure: plot.title.text
    plot.title.text = 'Gapminder data for %d' % yr


# Make a slider object: slider
slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)



# Create a dropdown Select widget for the x data: x_select
x_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='fertility',
    title='x-axis data'
)

# Attach the update_plot callback to the 'value' property of x_select
x_select.on_change('value', update_plot)

# Create a dropdown Select widget for the y data: y_select
y_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='life',
    title='y-axis data'
)

# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value', update_plot)



# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider, x_select, y_select), plot)

# Add the plot to the current document and add a title
curdoc().add_root(layout)
curdoc().title = 'Gapminder'

#Show layout
show(layout)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: region [renderer: GlyphRenderer(id='84216f70-ff8c-4011-87e7-385fe88a503f', ...)]
